In [1]:
using LinearAlgebra, Statistics, Roots, Optim, Plots, Random, Distributions

number = 1000
x = rand(Uniform(0,1),number)
ϵ = rand(Normal(0,0.5),1)
y = 2 .+ 1.2 .* x .+ ϵ
observations =[]
for i in 1:number
    push!(observations,(x[i],y[i]))
end


In [ ]:
function kdeCV(λ,i,observations)
    x,y = observations[i]
    newobservations = copy(observations)
    deleteat!(newobservations,i)
    kde(λ,x,y,newobservations)
end
xs = 0:1/2^3:20
ys = 0:1/2^3:10
D(u) = abs(u) < 1 ? 70/81*(1-abs(u)^3)^3 : 0 
D(λ,u) = 1/λ*D(u/λ) 
K(λ,x,y) = D(λ,x) * D(λ,y) 
kde(λ,x,y,observations) = sum(K(λ,x-Xi,y-Yi) for (Xi,Yi) in observations)/length(observations)


J(λ) = sum([kde(λ,x,y,observations)^2 for x=xs,y=ys])*step(xs)*step(ys) -
        2/length(observations)*sum(kdeCV(λ,i,observations) for i=1:length(observations))
λ_best_cv = optimize(λ->J(first(λ)),[1.0],BFGS())